In [1]:
import os
import random
import tensorflow as tf
import numpy as np

data_folder = "./data/train"

images = []
labels = []
categories = {"cat": 0, "dog": 1}

all_images = os.listdir(data_folder)
random.shuffle(all_images)

for file in all_images:
    if not file.endswith('.jpg'):
        continue
    image_path = os.path.join(data_folder, file)
    label = file.split(".")[0]
    images.append(image_path)
    labels.append(categories[label])
    
print(images[:3])
print(labels[:3])

['./data/train/cat.2536.jpg', './data/train/dog.10981.jpg', './data/train/cat.9547.jpg']
[0, 1, 0]


In [2]:
#How to use the Inception V3 model
some_images = images[:5]
with tf.Session() as sess:
    inception_model = "./data/model/inception_v3.pb"

    #Loading inception model from file
    with tf.gfile.FastGFile(inception_model, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

    #Find the output layer we are looking for
    LAST_LAYER = 'pool_3:0'
    tensor = sess.graph.get_tensor_by_name(LAST_LAYER)

    #Create features for all our images
    features = []
    for image in some_images:
        image_data = tf.gfile.FastGFile(image, 'rb').read()
        feature = sess.run(tensor, {'DecodeJpeg/contents:0': image_data})
        features.append(feature.flatten())
        
features = np.asarray(features)
print(features.shape)
print(features[0])

(5, 2048)
[ 0.51140302  0.04878923  0.18899484 ...,  0.0384578   0.26158404
  0.22986382]


In [3]:
inception_features = "./data/features_1000.txt"

images = []
features = []
labels = []
with open(inception_features, 'r') as f:   
    for line in f.readlines():
        values = line.rstrip().split(',')
        image = values[0]
        np_values = values[1:2049]
        
        images.append(image)
        features.append(np.asarray(np_values).flatten())

print(images[0])
print(features[0].shape)


dog.11508.jpg
(2048,)


In [4]:
training_size = 800
test_size = 200
total_size = training_size + test_size

In [5]:
labels = []
for image in images:
    label = image.split(".")[0]
    labels.append(categories[label])
    
print(images[0])
print(labels[0])

dog.11508.jpg
1


In [6]:
X = np.asarray(features)
Y = np.zeros([total_size, 2])

#one hot encoding
for i, label in enumerate(labels):
    Y[i][label] = 1
    
    
print(X.shape)
print(Y.shape)

(1000, 2048)
(1000, 2)


In [7]:
X_train = X[:training_size]
Y_train = Y[:training_size]

X_test = X[training_size:]
Y_test = Y[training_size:]

In [2]:
import tensorflow as tf
tf.reset_default_graph()

import tflearn
from tflearn.data_utils import shuffle
from tflearn.layers.core import input_data, fully_connected
from tflearn.layers.estimator import regression

network = input_data(shape=[None, 2048])


# Fully-connected 512 node neural network
network = fully_connected(network, 512, activation='relu')

# Fully-connected neural network with two outputs
network = fully_connected(network, 2, activation='softmax')

network = regression(network, optimizer='adam',
                     loss='categorical_crossentropy',
                     learning_rate=0.001)

# Wrap the network in a model object
model = tflearn.DNN(network, tensorboard_verbose=0)

# Train it!
model.fit(X_train, Y_train, n_epoch=10, shuffle=True, validation_set=0.1,
          show_metric=True, batch_size=50,
          run_id='dog-cat-inc')



hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


NameError: name 'X_train' is not defined

In [9]:
predictions = model.predict(X_test)

pred_label = np.argmax(predictions, axis=1)
real_label = np.argmax(Y_test, axis=1)

accuracy = np.sum(pred_label == real_label)/test_size

print("{}%".format(accuracy*100))

99.5%
